# Pricing of United States Treasury Notes and Bills

## Background
Treasury notes, also known as T-notes, are financial instruments that offer a fixed interest rate every six months until they mature. These instruments are available for 2, 3, 5, 7, and 10 years and can be priced above or below their face (par) value. Upon maturity, lenders are entitled to the full par value of the note. T-notes are considered coupon debt instruments, meaning lenders receive regular interest payments based on a coupon rate throughout the note’s lifespan. 

Treasury bonds, like T-notes, are debt instruments that pay a fixed interest rate every six months. Unlike T-notes, however, Treasury bonds are long-term U.S. Treasury debt instruments with terms of 20 or 30 years. Once a bond reaches maturity, the bondholder receives its face value.  Notes and bonds can be held until maturity or sold before that time.

### Pricing
Let the term of a bond (or note) be T-years with semiannual ($\lambda = 2$) coupon payments per year; $N = \lambda{T}$ coupon payments over the bond term. Further, let $\bar{c}$ denote the annualized coupon rate, and $\bar{r}$ denote the annual effective market interest rate. Then, the _fair price_ for the bond (or note) $V_{B}$ is the present value of coupon payments $C$ plus the discounted face (par) value $V_{P}$ of the bond:

$$
V_{B} = \frac{V_{P}}{\left(1+i\right)^{N}}+\sum_{j=1}^{N}\frac{C}{\left(1+i\right)^{j}}
$$

The coupon payment $C=\left(\bar{c}/\lambda\right)\cdot{V_{P}}$ is set when the bond is purchased, and the interest rate $i=\bar{r}/\lambda$ varies with the market. The contract between the U.S. government, the issuer, and the bondholder (you) includes the bond’s coupon rate, maturity date, and face (par) value.

## Objectives
Let's use public quotation data from the [Wall Street Journal](https://www.wsj.com/market-data/bonds/treasuries) to test the pricing formula above (and our codes to calculte $V_{B}$). Next, we'll do a [Yield to Maturity (YTM)](https://en.wikipedia.org/wiki/Yield_to_maturity) calculation for notes and bonds of different durations.

### Tasks and Strategies
1. Validate the prices calculated using our implementation of the pricing formula above with historical data for bonds and notes of different durations. Further, show that longer term bonds are more sensitive to changes in the market interest rate compared to shorter duration instruments. 
1. Using our validated bond/note code, compute the [Yield to Maturity (YTM)](https://en.wikipedia.org/wiki/Yield_to_maturity) for bonds of different durations. Show how the parameters in the bond pricing equation influence the yeild to majurity. 

## Setup
In the following code blocks, we set up the computational environment by including the `Include.jl` file. The `Include.jl` file loads external packages, functions that we will use in this exercise, and custom types to model the components of our problem.

In [1]:
include("Include.jl");

## a) Validate the Bond Pricing Formula
Let's consider a few examples of coupon Bond pricing with our implementation of the pricing formula should above.  

#### Example 1: A 5-year treasury note with semi-annual coupon payments
Compute the price of a 5-year treasury note with semi-annual coupons with a coupon rate of 8% and a market rate of interest of 7%. The face value of the note is $V_{P}=$10,000 USD. We expect a price of $V_{B}$ = 10,415.83 USD.

In [21]:
# Setup parameters for this example
Vₚ = 100.0;
T = 5.0
r = 0.07
c = 0.08
λ = 2

# Build model, and compute the price
model = build(MyUSTreasuryCouponSecurityModel, (
            par = Vₚ, T = T, rate = r, coupon = c, λ = λ
        )) |> (model -> price(model, Vₚ = Vₚ));

# pretty table -
example_1_data_array = Array{Any,2}(undef, 2,2);
example_1_data_array[1,1] = "Reported"
example_1_data_array[1,2] = "10,415.83"
example_1_data_array[2,1] = "Calculated"
example_1_data_array[2,2] = 10000.0*model.price*(1/100)

example_1_header_data = (["","Value (USD)"]);
pretty_table(example_1_data_array, header=example_1_header_data);

┌────────────┬─────────────┐
│            │ Value (USD) │
├────────────┼─────────────┤
│   Reported │   10,415.83 │
│ Calculated │     10415.8 │
└────────────┴─────────────┘


##### Cashflow table for Example 1

In [9]:
cashflow_data_table = Array{Any,2}(undef,11,5);
cfd = model.cashflow;
number_of_periods = length(cfd);
C = (c/λ)*Vₚ
rᵢ = (r/λ);

counter = 1;
cumulative_sum = 0.0
for i ∈ 0:(number_of_periods-1)
    
    # compute the discount rate -
    𝒟ᵢ = (1+rᵢ)^i
    
    # update the cumulative sum 
    cumulative_sum += cfd[i];
    
    if (i == 0)
        cashflow_data_table[counter, 1] = i;
        cashflow_data_table[counter, 2] = cfd[i];
        cashflow_data_table[counter, 3] = cfd[i];
        cashflow_data_table[counter, 4] = cumulative_sum;
        # cashflow_data_table[counter, 5] = (1/𝒟ᵢ)*model.price
        cashflow_data_table[counter, 5] = (1/𝒟ᵢ)*model.price
    elseif (i == (number_of_periods-1))
        cashflow_data_table[counter, 1] = i;
        cashflow_data_table[counter, 2] = C + Vₚ
        cashflow_data_table[counter, 3] = cfd[i];
        cashflow_data_table[counter, 4] = cumulative_sum;
        # cashflow_data_table[counter, 5] = (1/𝒟ᵢ)*model.price 
        cashflow_data_table[counter, 5] = (1/𝒟ᵢ)*model.price
     else
        cashflow_data_table[counter, 1] = i;
        cashflow_data_table[counter, 2] = C
        cashflow_data_table[counter, 3] = cfd[i];
        cashflow_data_table[counter, 4] = cumulative_sum;
        # cashflow_data_table[counter, 5] = (1/𝒟ᵢ)*model.price
        cashflow_data_table[counter, 5] = (1/𝒟ᵢ)*model.price
    end
    counter += 1;
end

# cashflow_data_table[end,1] = "Total"
# cashflow_data_table[end,2] = sum(cashflow_data_table[1:end-1, 2])
# cashflow_data_table[end,3] = sum(cashflow_data_table[1:end-1, 3])
# cashflow_data_table[end,4] = sum(cashflow_data_table[1:end-1, 4])
# cashflow_data_table[end,5] = sum(cashflow_data_table[1:end-1, 5])

cashflow_table_header = (["Period","Cashflow (CF) (USD)", "Discounted Cashflow (DCF) (PUSD)", "Cumulative DCF (PUSD)", "Alternative"]);
pretty_table(cashflow_data_table, header = cashflow_table_header)

┌────────┬─────────────────────┬──────────────────────────────────┬───────────────────────┬─────────────┐
│ Period │ Cashflow (CF) (USD) │ Discounted Cashflow (DCF) (PUSD) │ Cumulative DCF (PUSD) │ Alternative │
├────────┼─────────────────────┼──────────────────────────────────┼───────────────────────┼─────────────┤
│      0 │            -10415.8 │                         -10415.8 │              -10415.8 │     10415.8 │
│      1 │               400.0 │                          386.473 │              -10029.4 │     10063.6 │
│      2 │               400.0 │                          373.404 │              -9655.95 │     9723.29 │
│      3 │               400.0 │                          360.777 │              -9295.18 │     9394.48 │
│      4 │               400.0 │                          348.577 │               -8946.6 │     9076.79 │
│      5 │               400.0 │                          336.789 │              -8609.81 │     8769.85 │
│      6 │               400.0 │              

#### Example 2: A 2-year treasury note with annual coupon payments
Compute the price of a 2-year treasury note with annual coupons with a coupon rate of 7% and a market rate of interest of 8%. The face value of the note is $V_{P}=$100 USD. We expect a price of $V_{B}$ = 98.22 USD. This example was repoduced from [FINA 635, Department of Finance at the C.T. Bauer College of Business, University of Houston](https://www.bauer.uh.edu/nlangberg/teaching_files/Fall2010/class%205_2008sum.pdf)

In [22]:
# Setup parameters for this example
Vₚ = 100.0;
T = 2.0
r = 0.08
c = 0.07
λ = 1

# Build model, and compute the price
model = build(MyUSTreasuryCouponSecurityModel, (
            par = Vₚ, T = T, rate = r, coupon = c, λ = λ
        )) |> (x -> price(x, Vₚ = Vₚ));

# pretty table -
example_1_data_array = Array{Any,2}(undef, 2,2);
example_1_data_array[1,1] = "Reported"
example_1_data_array[1,2] = "98.22"
example_1_data_array[2,1] = "Calculated"
example_1_data_array[2,2] = model.price

example_1_header_data = (["","Value (USD)"]);
pretty_table(example_1_data_array, header=example_1_header_data);

┌────────────┬─────────────┐
│            │ Value (USD) │
├────────────┼─────────────┤
│   Reported │       98.22 │
│ Calculated │     98.2167 │
└────────────┴─────────────┘


#### Example 3: Comparision of the calculated 30-year Treasury bond price with historical data

In [27]:
# Setup parameters for this example
Vₚ = 100.0;
T = 30.0
r = 0.0794
c = 0.07875
λ = 2

# Build model, and compute the price
model = build(MyUSTreasuryCouponSecurityModel, (
            par = Vₚ, T = T, rate = r, coupon = c, λ = λ
        )) |> (x -> price(x, Vₚ = Vₚ));

# pretty table -
example_1_data_array = Array{Any,2}(undef, 2,2);
example_1_data_array[1,1] = "Reported"
example_1_data_array[1,2] = "98.22"
example_1_data_array[2,1] = "Calculated"
example_1_data_array[2,2] = model.price

example_1_header_data = (["","Value (USD)"]);
pretty_table(example_1_data_array, header=example_1_header_data);

┌────────────┬─────────────┐
│            │ Value (USD) │
├────────────┼─────────────┤
│   Reported │       98.22 │
│ Calculated │     99.2605 │
└────────────┴─────────────┘
